# 1.1 Imports

In [44]:
# import libraries

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

from pyspark.sql import SparkSession, Window
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col, concat, count, desc, \
asc, explode, lit, min, max, split, stddev, udf, isnan, when, rank, \
log, sqrt, cbrt, exp, round
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, \
LogisticRegressionModel, RandomForestClassifier, \
RandomForestClassificationModel, GBTClassifier, \
GBTClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, \
PCA, RegexTokenizer, Tokenizer, StandardScaler, StopWordsRemover, \
StringIndexer, VectorAssembler, MaxAbsScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# create parksession

spark = SparkSession.builder.master('local') \
        .appName('Sparkify').getOrCreate()

21/07/20 16:43:10 WARN Utils: Your hostname, Yats-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.3 instead (on interface en0)
21/07/20 16:43:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/20 16:43:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# load data

path = 'mini_sparkify_event_data.json'
df = spark.read.json(path)
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [5]:
# check schema
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# 1.2 Missing Values

# 1.2.1 Observations

In [6]:
# total number of rows
print(f'total number of rows: {df.count()}')

total number of rows: 286500


In [7]:
# number of missing values

for c in df.columns:
    df.select(
        [count(when(df[c].isNull(),True))]
    ).show()

+-----------------------------------------------+
|count(CASE WHEN (artist IS NULL) THEN true END)|
+-----------------------------------------------+
|                                          58392|
+-----------------------------------------------+

+---------------------------------------------+
|count(CASE WHEN (auth IS NULL) THEN true END)|
+---------------------------------------------+
|                                            0|
+---------------------------------------------+

+--------------------------------------------------+
|count(CASE WHEN (firstName IS NULL) THEN true END)|
+--------------------------------------------------+
|                                              8346|
+--------------------------------------------------+

+-----------------------------------------------+
|count(CASE WHEN (gender IS NULL) THEN true END)|
+-----------------------------------------------+
|                                           8346|
+-----------------------------------------

# 1.2.2 Missing Values in userId

In [8]:
# from above findings, 0 rows in userId is NULL
# so we check for samples

df.select(['userId']).drop_duplicates().orderBy(df['userId']).show(10)

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
+------+
only showing top 10 rows



In [9]:
# the first row is an empty string

In [10]:
# create a temporary view for running SQL queries
df.createOrReplaceTempView('df_table')

In [11]:
# display the page events of the user with empty string userId

spark.sql(
    '''
    SELECT DISTINCT page
    FROM df_table
    WHERE userId == ''
    '''
).show()

+-------------------+
|               page|
+-------------------+
|               Home|
|              About|
|Submit Registration|
|              Login|
|           Register|
|               Help|
|              Error|
+-------------------+



In [12]:
# display the page events of another user with non-empty string userId

spark.sql(
    '''
    SELECT DISTINCT page
    FROM df_table
    WHERE userId == 100001
    '''
).show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|         Thumbs Down|
|                Home|
|         Roll Advert|
|              Logout|
|Cancellation Conf...|
|            Settings|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
+--------------------+



In [13]:
# we can see that 
# the page events of empty string userId user has nearly no features page except
# 'Submit Registration' which might indicate that it is a user with incomplete registration

In [14]:
# drop the row with empty userId
df = df.filter(df['userId'] != '')

# 1.3 Feature Engineering

# 1.3.1 Overview of Numerical Features

In [15]:
# number of unique users

unique_users = df.select('userId').drop_duplicates().count()
print(unique_users)

225


In [16]:
# numerical columns
num_cols = ['itemInSession', 'registration', 'status', 'ts', 'length']

In [17]:
# describe numerical columns
df.select(num_cols).describe().show()

+-------+------------------+--------------------+------------------+--------------------+-----------------+
|summary|     itemInSession|        registration|            status|                  ts|           length|
+-------+------------------+--------------------+------------------+--------------------+-----------------+
|  count|            278154|              278154|            278154|              278154|           228108|
|   mean|114.89918174824018|1.535358834084427...|209.10321620397335|1.540958915431871...|249.1171819778458|
| stddev|  129.851729399489| 3.291321616327586E9|30.151388851328214|1.5068287123306298E9|99.23517921058361|
|    min|                 0|       1521380675000|               200|       1538352117000|          0.78322|
|    max|              1321|       1543247354000|               404|       1543799476000|       3024.66567|
+-------+------------------+--------------------+------------------+--------------------+-----------------+



In [18]:
# 'status' is HTTP status

header = 'status'
df.groupBy(header).count() \
.withColumn('%', round((col('count')/df.count())*100,2)) \
.orderBy('%', ascending=False).show()

+------+------+-----+
|status| count|    %|
+------+------+-----+
|   200|254718|91.57|
|   307| 23184| 8.33|
|   404|   252| 0.09|
+------+------+-----+



In [19]:
# 'status' should be classified as categorical feature

# 1.3.2 Overview of Categorical Features

In [20]:
# catagorical columns
cat_cols = [
    'artist', 'auth', 'firstName', 'lastName', 
    'gender', 'level', 'location','method', 
    'page', 'song', 'userAgent'
]

In [21]:
# we only observe the features that can be classified

In [22]:
# values count of 'auth'

header = 'auth'
df.groupBy(header).count() \
.withColumn('%', round((col('count')/df.count())*100,2)) \
.orderBy('%', ascending=False).show()

+---------+------+-----+
|     auth| count|    %|
+---------+------+-----+
|Logged In|278102|99.98|
|Cancelled|    52| 0.02|
+---------+------+-----+



In [23]:
# values count of 'gender'

header = 'gender'
df.groupBy(header).count() \
.withColumn('%', round((col('count')/df.count())*100,2)) \
.orderBy('%', ascending=False).show()

+------+------+-----+
|gender| count|    %|
+------+------+-----+
|     F|154578|55.57|
|     M|123576|44.43|
+------+------+-----+



In [24]:
# values count of 'level'

header = 'level'
df.groupBy(header).count() \
.withColumn('%', round((col('count')/df.count())*100,2)) \
.orderBy('%', ascending=False).show()

+-----+------+-----+
|level| count|    %|
+-----+------+-----+
| paid|222433|79.97|
| free| 55721|20.03|
+-----+------+-----+



In [25]:
# first 5 values count of 'location'

header = 'location'
df.groupBy(header).count() \
.withColumn('%', round((col('count')/df.count())*100,2)) \
.orderBy('%', ascending=False).limit(5).show()

+--------------------+-----+-----+
|            location|count|    %|
+--------------------+-----+-----+
|Los Angeles-Long ...|30131|10.83|
|New York-Newark-J...|23684| 8.51|
|Boston-Cambridge-...|13873| 4.99|
|Houston-The Woodl...| 9499| 3.42|
|Charlotte-Concord...| 7780|  2.8|
+--------------------+-----+-----+



In [26]:
# values count of 'method'

header = 'method'
df.groupBy(header).count() \
.withColumn('%', round((col('count')/df.count())*100,2)) \
.orderBy('%', ascending=False).show()

+------+------+-----+
|method| count|    %|
+------+------+-----+
|   PUT|257818|92.69|
|   GET| 20336| 7.31|
+------+------+-----+



In [27]:
# unique values of 'page'

header = 'page'
print([x[header] for x in df.select(header).drop_duplicates().collect()])

['Cancel', 'Submit Downgrade', 'Thumbs Down', 'Home', 'Downgrade', 'Roll Advert', 'Logout', 'Save Settings', 'Cancellation Confirmation', 'About', 'Settings', 'Add to Playlist', 'Add Friend', 'NextSong', 'Thumbs Up', 'Help', 'Upgrade', 'Error', 'Submit Upgrade']


# 1.3.3 Define Churn

In [28]:
# churn can be detected by the appearance of 'Cancellation Confirmation' in column 'page'

In [29]:
# define function
churn = udf(lambda x: 1 if 'Cancellation Confirmation' in x else 0, IntegerType())

# create feature of churn
df = df.withColumn('churned', churn('page'))

In [30]:
# distribution of 'churned' over the no of unique users

df.groupBy('userId').agg({'churned':'sum'}).select(avg('sum(churned)')).show()

+------------------+
| avg(sum(churned))|
+------------------+
|0.2311111111111111|
+------------------+



In [31]:
# 23.1% of users have churned

In [32]:
# create feature of churned users
df = df.withColumn('user_churn', max('churned').over(Window.partitionBy('UserId')))

In [33]:
df.head(1)

[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churned=0, user_churn=0)]

In [34]:
# convert timestamp format
format_ts = udf(lambda x: datetime.fromtimestamp(x / 1000.0).strftime('%m-%d-%Y %H:%M:%S'))

# convert 'registration' column
df = df.withColumn('registration_datetime', format_ts('registration'))

# convert 'ts' column
df = df.withColumn('event_datetime', format_ts('ts'))

In [35]:
# observe first item
df.head(1)

[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churned=0, user_churn=0, registration_datetime='09-27-2018 10:45:40', event_datetime='10-08-2018 20:58:54')]

In [37]:
# display first 10 unique userId with selected columns

df.select(['userId', 'firstName', 'lastName', 'level', 'registration_datetime', 'user_churn']).drop_duplicates(['userId']).show(10)

+------+---------+---------+-----+---------------------+----------+
|userId|firstName| lastName|level|registration_datetime|user_churn|
+------+---------+---------+-----+---------------------+----------+
|100010| Darianna|Carpenter| free|  09-27-2018 10:45:40|         0|
|200002|  Dominic|   Torres| free|  09-07-2018 05:38:26|         0|
|   125|    Mason|     Hart| free|  08-02-2018 04:58:59|         1|
|   124|   Nicole|     Beck| paid|  07-22-2018 09:52:15|         0|
|    51|    Ethan|  Johnson| paid|  09-28-2018 04:43:07|         1|
|     7|   Maddox|  Johnson| free|  09-11-2018 20:06:16|         0|
|    15|    Panav|    Myers| paid|  09-30-2018 00:37:25|         0|
|    54|    Alexi|   Warren| paid|  07-25-2018 09:37:42|         1|
|   155|   Camila| Calderon| free|  11-05-2018 06:27:50|         0|
|100014|  Rodrigo|    Carey| paid|  08-28-2018 01:04:03|         1|
+------+---------+---------+-----+---------------------+----------+
only showing top 10 rows



# 1.4 Exploratory Data Analysis

In [ ]:
# andy p w yu
# 1. churn uers by gender
# 2. paid/free customers churn 
# 3. no of songs played per session
# 4. membership period (time since registration) [critical period to make users stay]
# 5. no of friends 

## angang li
# 6. paid/free churn ratio
# 7. page event (12 boxplots!) don't need distribution
# 8. thumbsup/down
# 9. locations [geographically different user habits]

# advertisement/ marketing/ strategy
# 1, 2

# user experience/ service
# 3, 4, 5, 6, 7, 8, 

# clustering/ segmentation
# 9

# 1.4.1 Advertisement/ marketing

# 1.4.1.1 Genders

In [40]:
# gender distribution

df_gender_pd = df.drop_duplicates(['userId', 'gender'])\
    .groupby(['user_churn', 'gender']).count().sort('user_churn').toPandas()
df_gender_pd

,user_churn,gender,count
0,0,M,89
1,0,F,84
2,1,F,20
3,1,M,32


In [41]:
# display in bar chart

plt.title('number of users churn with respect to genders')
sns.barplot(x = 'user_churn', y = 'count', data = df_gender_pd, hue = 'gender')

NameError: name 'plt' is not defined

In [ ]:
# END